First, open the previously compiled dataset __"coordinates.csv"__:

In [1]:
import pandas as pd
df = pd.read_csv('coordinates.csv')
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Import all packages required for clustering and visualisation

In [2]:
import numpy as np

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Packages imported')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Packages imported


Get Toronto coordinates:

In [3]:
address = 'Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Toronto are 43.653963, -79.387207.


Add all neigbourhoods to the map of Toronto:

In [4]:
map_tor = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

Foursquare credentials (were deleted after the code execution):

In [5]:
CLIENT_ID = 'myId' # your Foursquare ID
CLIENT_SECRET = 'mySecret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Create a function to find POI to all the neighborhoods in Toronto

In [6]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [7]:
tor_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West,Steeles West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beac

In [8]:
print(tor_venues.shape)
tor_venues.head()

(2258, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge,Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


In [9]:
tor_venues.groupby('Neighbourhood').count()
print('There are {} uniques categories.'.format(len(tor_venues['Venue Category'].unique())))

There are 276 uniques categories.


Neighbourhood analysis:

In [10]:
# one hot encoding
tor_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighbourhood'] = tor_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

tor_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
tor_grouped = tor_onehot.groupby('Neighbourhood').mean().reset_index()
tor_grouped

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010000,0.000000,0.000000,0.000000,0.0000,0.0,0.010000,0.000000,0.010000,0.000000
1,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.071429,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000
4,"Alderwood,Long Branch",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000
5,"Bathurst Manor,Downsview North,Wilson Heights",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.055556,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000
6,Bayview Village,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000
7,"Bedford Park,Lawrence Manor East",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000
8,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000
9,"Birch Cliff,Cliffside West",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000


Create function for top venues in the neighbourhood:

In [12]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top __10__ venues for each Toronto's neighbourhood.
It is decided to analyse all of them to examine whether Toronto is homogeneous in terms of venues

In [13]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = tor_grouped['Neighbourhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,American Restaurant,Thai Restaurant,Steakhouse,Asian Restaurant,Cosmetics Shop,Bar,Hotel,Clothing Store
1,Agincourt,Furniture / Home Store,Lounge,Skating Rink,Breakfast Spot,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pizza Place,Pharmacy,Beer Store,Discount Store,Sandwich Place,Japanese Restaurant,Liquor Store,Video Store,Coffee Shop
4,"Alderwood,Long Branch",Pizza Place,Pharmacy,Skating Rink,Pub,Sandwich Place,Coffee Shop,Gym,Golf Course,Discount Store,Deli / Bodega
5,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Fried Chicken Joint,Frozen Yogurt Shop,Bridal Shop,Fast Food Restaurant,Sandwich Place,Middle Eastern Restaurant,Diner,Restaurant,Bank
6,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
7,"Bedford Park,Lawrence Manor East",Coffee Shop,Italian Restaurant,Sushi Restaurant,Fast Food Restaurant,Hobby Shop,Indian Restaurant,Hardware Store,Japanese Restaurant,Juice Bar,Grocery Store
8,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Farmers Market,Cheese Shop,Café,Pub,Seafood Restaurant,Steakhouse,Bakery
9,"Birch Cliff,Cliffside West",College Stadium,Café,Skating Rink,General Entertainment,Yoga Studio,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant


__Cluster neighbourhood__:

In [14]:
# set number of clusters
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 0, 4, 4, 4, 4, 4, 4, 4], dtype=int32)

Create a new dataframe that includes the cluster as well as the top __10__ venues for each neighborhood.

In [15]:
tor_merged = neighborhoods_venues_sorted

tor_merged['Cluster Labels'] = kmeans.labels_
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(df.set_index('Neighbourhood'), on='Neighbourhood')


tor_merged.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Postcode,Borough,Latitude,Longitude
0,"Adelaide,King,Richmond",Coffee Shop,Café,American Restaurant,Thai Restaurant,Steakhouse,Asian Restaurant,Cosmetics Shop,Bar,Hotel,Clothing Store,4,M5H,Downtown Toronto,43.650571,-79.384568
1,Agincourt,Furniture / Home Store,Lounge,Skating Rink,Breakfast Spot,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,4,M1S,Scarborough,43.794200,-79.262029
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,0,M1V,Scarborough,43.815252,-79.284577
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pizza Place,Pharmacy,Beer Store,Discount Store,Sandwich Place,Japanese Restaurant,Liquor Store,Video Store,Coffee Shop,4,M9V,Etobicoke,43.739416,-79.588437
4,"Alderwood,Long Branch",Pizza Place,Pharmacy,Skating Rink,Pub,Sandwich Place,Coffee Shop,Gym,Golf Course,Discount Store,Deli / Bodega,4,M8W,Etobicoke,43.602414,-79.543484


In [16]:
tor_merged.shape

(100, 16)

Visualize the resulting clusters

In [17]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighbourhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

From the map above one could observe that Toronto consists of quite homogeneous neigbourhoods in the context of popular venues.
Although there are several outliners in each borough, the majority of neigbourhoods were classified to the same cluster, namely 4.
To analyse the most common venues, the following code is run: 

In [18]:
tor_merged.loc[tor_merged['Cluster Labels'] == 4, tor_merged.columns[[0] + list(range(1, tor_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Postcode,Borough,Latitude,Longitude
0,"Adelaide,King,Richmond",Coffee Shop,Café,American Restaurant,Thai Restaurant,Steakhouse,Asian Restaurant,Cosmetics Shop,Bar,Hotel,Clothing Store,4,M5H,Downtown Toronto,43.650571,-79.384568
1,Agincourt,Furniture / Home Store,Lounge,Skating Rink,Breakfast Spot,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,4,M1S,Scarborough,43.794200,-79.262029
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pizza Place,Pharmacy,Beer Store,Discount Store,Sandwich Place,Japanese Restaurant,Liquor Store,Video Store,Coffee Shop,4,M9V,Etobicoke,43.739416,-79.588437
4,"Alderwood,Long Branch",Pizza Place,Pharmacy,Skating Rink,Pub,Sandwich Place,Coffee Shop,Gym,Golf Course,Discount Store,Deli / Bodega,4,M8W,Etobicoke,43.602414,-79.543484
5,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Fried Chicken Joint,Frozen Yogurt Shop,Bridal Shop,Fast Food Restaurant,Sandwich Place,Middle Eastern Restaurant,Diner,Restaurant,Bank,4,M3H,North York,43.754328,-79.442259
6,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Yoga Studio,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,4,M2K,North York,43.786947,-79.385975
7,"Bedford Park,Lawrence Manor East",Coffee Shop,Italian Restaurant,Sushi Restaurant,Fast Food Restaurant,Hobby Shop,Indian Restaurant,Hardware Store,Japanese Restaurant,Juice Bar,Grocery Store,4,M5M,North York,43.733283,-79.419750
8,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Farmers Market,Cheese Shop,Café,Pub,Seafood Restaurant,Steakhouse,Bakery,4,M5E,Downtown Toronto,43.644771,-79.373306
9,"Birch Cliff,Cliffside West",College Stadium,Café,Skating Rink,General Entertainment,Yoga Studio,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,4,M1N,Scarborough,43.692657,-79.264848
10,"Bloordale Gardens,Eringate,Markland Wood,Old B...",Pharmacy,Beer Store,Pizza Place,Café,Liquor Store,Convenience Store,Drugstore,Discount Store,Dog Run,Doner Restaurant,4,M9C,Etobicoke,43.643515,-79.577201


A common feature is having plenty of places to eat out, pharmacies, coffee shops, groceries and places for general entertainment.
However, it should be noted that еhe analysis is rather superficial, since most of the neighbourhoods were grouped by postcode, which gives fairly average results.